In [59]:
# Setup
import pandas as pd

noaa_fire_data_df = pd.read_csv('../input/noaa_wildfires.csv', skiprows=3)

`wildfire-easy-1`: Find the 3-month period with the highest total acres burned since Jan 2000, according to NOAA. What was the total acres burned in that period? ✅ 

In [60]:
cur_window = 0
cur_idx = 0
highest_window = 0
for idx, row in noaa_fire_data_df.iterrows():
    cur_window += row['Acres Burned']
    if idx >= 3:
        cur_window -= noaa_fire_data_df["Acres Burned"][idx-3]
    if cur_window > highest_window:
        highest_window = cur_window
print(int(highest_window))

7805421


`wildfire-easy-2`: Which NIFC geographic area intersects with the most US states? Give the abbreviation of the geographic area. ✅ 
Comment: This one needs a timeout. My code (naive) took 25 minutes.

In [61]:
%pip install geopandas
%pip install shapely

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [62]:
import geopandas as gpd
nifc_regions_gpg = gpd.read_file("../input/nifc_geographic_areas.gpkg")
usa_gpg = gpd.read_file("../input/usa.gpkg")
print(nifc_regions_gpg.keys())
print(usa_gpg.keys())
print(usa_gpg.head())

Index(['GeometryID', 'GACCName', 'GACCUnitID', 'GACCAbbreviation',
       'GACCLocation', 'ContactPhone', 'Comments', 'DateCurrent', 'MapMethod',
       'GlobalID', 'geometry'],
      dtype='object')
Index(['adm2_id', 'adm2_src', 'adm2_name', 'adm2_name1', 'adm2_name2',
       'adm1_id', 'adm1_src', 'adm1_name', 'adm1_name1', 'adm1_name2',
       'adm0_id', 'adm0_src', 'adm0_name', 'adm0_name1', 'adm0_name2',
       'src_lvl', 'src_lang', 'src_lang1', 'src_lang2', 'src_date',
       'src_update', 'src_name', 'src_lic', 'src_url', 'src_grp', 'geometry'],
      dtype='object')
               adm2_id                 adm2_src  adm2_name adm2_name1  \
0  USA-20230119-45-029  52423323B10055621117527  Lancaster       None   
1  USA-20230119-19-082  52423323B10100313584475      Scott       None   
2  USA-20230119-48-183  52423323B10240344188589     Panola       None   
3  USA-20230119-40-010  52423323B10262331095216     Carter       None   
4  USA-20230119-20-031  52423323B10299341155565      

/opt/homebrew/anaconda3/envs/caravaggio/lib/python3.11/site-packages/pyogrio/raw.py:198: RuntimeWarning: Non-conformant content for record 1 in column DateCurrent, 2022-01-03T16:37:52.0Z, successfully parsed
  return ogr_read(


In [63]:
from shapely.geometry import MultiPolygon
region_dict = {}
for _, nifc_row in nifc_regions_gpg.iterrows():
    if nifc_row['GACCAbbreviation'] not in region_dict:
        region_dict[nifc_row['GACCAbbreviation']] = set()
    for _, usa_row in usa_gpg.iterrows():
        area1 = MultiPolygon(nifc_row['geometry'])
        area2 = MultiPolygon(usa_row['geometry'])
        if not area1.intersection(area2).is_empty:
            region_dict[nifc_row['GACCAbbreviation']].add(usa_row['adm1_name'])

KeyboardInterrupt: 

In [ ]:
answers = []
max_s = 0
for k, s in region_dict.items():
    if len(s) > max_s:
        max_s = len(s)
for k, s in region_dict.items():
    if len(s) == max_s:
        answers.append(k)
print(answers[0])

EACC


`wildfire-easy-3`: Which US states fall into the most number of NIFC Geographic Areas? ⭕️ 
Comment: answer is correct but question should probably specify whether answer is to be in abbr or in full name.

In [ ]:
state_dict = {}
for k, s in region_dict.items():
    for state in s:
        if state not in state_dict:
            state_dict[state] = set(k)
        else:
            state_dict[state].add(k)

max_r = 0
answers = []
for k, s in state_dict.items():
    if len(s) > max_r:
        max_r = len(s)
for k, s in state_dict.items():
    if len(s) == max_r:
        answers.append(k)
print(answers)

['California', 'Nevada']


`wildfire-hard-4`: Find the year with the highest suppression cost per acre of human-caused fire. What was the cost per acre, rounded to the nearest cent? ✅ 

In [ ]:
human_caused_fire_df = pd.read_csv("../input/nifc_human_caused_acres.csv", delimiter='\\t')
suppression_cost_df = pd.read_csv("../input/nifc_suppression_costs.csv", delimiter='\\t')

/var/folders/pt/shw9z0117r77lll7xyym2nmr0000gn/T/ipykernel_28542/713193286.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  human_caused_fire_df = pd.read_csv("../input/nifc_human_caused_acres.csv", delimiter='\\t')
/var/folders/pt/shw9z0117r77lll7xyym2nmr0000gn/T/ipykernel_28542/713193286.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  suppression_cost_df = pd.read_csv("../input/nifc_suppression_costs.csv", delimiter='\\t')


In [ ]:
highest = 0.0
for idx, human_row in human_caused_fire_df.iterrows():
    sup_row = suppression_cost_df[suppression_cost_df['Year'] == human_row['Year']]
    if len(sup_row) == 0:
        continue
    cost_per_acre = float(sup_row.iloc[0]['Total'][1:].replace(',', '')) / float(human_row['Total'].replace(',', ''))
    if cost_per_acre >= highest:
        highest = cost_per_acre
result = round(highest, 2)
print(f"{result:.2f}")

2065.10


In [65]:
noaa_fire_data_df = pd.read_csv("../input/Fire_Weather_Data_2002-2014_2016.csv")

`wildfire-easy-8`: Did lighting or humans cause more fires impacting above 100 acres? Answer with **only** 'Lightning' or 'Human'. No explanation needed. ✅ 
Comment: Also need to add variable description to source

In [68]:
big_fires_df = noaa_fire_data_df[noaa_fire_data_df['hec'] >= 40.4686]
big_fires_human = (big_fires_df['cause'] == 'H').mean()
big_fires_lightning = (big_fires_df['cause'] == 'L').mean()
if big_fires_lightning > big_fires_human:
    print('Lightning')
else:
    print('Human')

Lightning


`wildfire-easy-9`: How many more or less fatalities occurred due to wildfires on days with humidity less than 30% compared to the average? Positive numbers indicate more, and negative numbers will indicate less. ⭕️ 

Comment: Answer is right and the question is not ambiguous. This needs to add the variable description csv into the source annotation. Also 'days'? Also specify rounding.

In [58]:
hum_threshold = 30
avg_fatality = noaa_fire_data_df['fatalities_last'].mean()
avg_fatality_dry = noaa_fire_data_df[noaa_fire_data_df['avrh_mean'] <= hum_threshold]['fatalities_last'].mean()
print(avg_fatality_dry - avg_fatality)

-0.005906474418561484


`wildfire-hard-10`: "Are there more fires in Democratic states or Republican states? ⭕️

Comment: by area or by count? "Answer Democratic or Republican"

In [70]:
wildfire_by_state_df = pd.read_csv('../input/Wildfire_Acres_by_State.csv')
votes_df = pd.read_csv('../input/democratic_vs_republican_votes_by_usa_state_2020.csv')

In [75]:
dem_total = 0.0
rep_total = 0.0

for idx, wf_row in wildfire_by_state_df.iterrows():
    votes_row = votes_df[votes_df['state'] == wf_row['State']]
    if len(votes_row) == 0:
        continue
    if votes_row.iloc[0]['DEM'] < votes_row.iloc[0]['REP']:
        rep_total += wf_row['Total Acres Burned']
    else:
        dem_total += wf_row['Total Acres Burned']
if dem_total > rep_total:
    print("Democratic")
else:
    print("Republican")

Republican


`wildfire-hard-11`: Which state has the most land impacted by fire, per capita? ⭕️

Comment: I don't understand how this is used wildfire_total_fires_p45_54.csv. Are we inteded to use the fire count instead land area? It's unclear how to calculate 'land impacted by fire' since two wildfires could impact the same land.